<a href="https://colab.research.google.com/github/antoniodragoc/Projeto-de-Graduacao-Antonio-Drago-Caetano/blob/main/3_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the needed packages
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Conv2D

from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns

import pandas as pd

from fast_ml.model_development import train_valid_test_split
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
folder = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/'
for filename in os.listdir(folder):
        print(filename)
        # Caminhos de treino e validaçção

        # Obs - no diretório dados4 os dados estão separados em 75% treino 15% validação e 10% teste
        path_train = folder+filename+'/train/'
        path_val = folder+filename+'/val/'

        batch_size = 20

        # Gerando os dados a partir das imagens
        train_datagen = ImageDataGenerator(    
            rotation_range = 90,
            vertical_flip = True,
            horizontal_flip = True,
            zoom_range=0.05,
            width_shift_range=0.2,
            height_shift_range=0.2,
            fill_mode = 'reflect',
            samplewise_center = True,
            samplewise_std_normalization = True
        )

        val_datagen = ImageDataGenerator(
            samplewise_center = True,
            samplewise_std_normalization = True
            )


        test_datagen = ImageDataGenerator(
            samplewise_center = True,
            samplewise_std_normalization = True
            )

        # dados de treino
        train_generator = train_datagen.flow_from_directory(
            path_train,
            target_size = (256, 256),
            batch_size= batch_size,
            shuffle=True,
            class_mode= 'categorical' 
        )

        y_true = train_generator.classes
        y_true

        #dados de validação
        val_generator = val_datagen.flow_from_directory(
            path_val,
            target_size = (256, 256),
            batch_size= batch_size,  
            shuffle=True,
            class_mode='categorical'     
        )

        val_generator.classes

        # Chamando a rede pré-treinada
        conv_model = ResNet50(
            #weights='imagenet', 
            include_top=False, 
            input_shape=(256,256,3))

        first_freeze_layers = 20 #number of the first layers to freeze
        for layer in conv_model.layers[0:first_freeze_layers]:
            layer.trainable = False

        teste = keras.layers.Flatten()(conv_model.output)

        # three hidden layers
        teste = keras.layers.Dropout(0.4)(teste)
        teste = keras.layers.Dense(128, activation='relu')(teste)
        teste = keras.layers.Dense(64, activation='relu')(teste)

        # final softmax layer with two categories
        predictions = keras.layers.Dense(2, activation='softmax')(teste)

        # creating the full model:
        full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

        #full_model.summary()

        #EARLY STOPPING ADICIONADO
        # https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
        callback = keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0,
            patience=15,
            verbose=0,
            mode="min",
            baseline=None,
            restore_best_weights=True
        )
        
        reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1,
                                                 patience=15)
        
        lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
            initial_learning_rate = 0.001, 
            first_decay_steps = 500,
            t_mul=2.4, 
            m_mul=1.3, 
            alpha=0.001, 
            name=None
        )

        # Compilando o modelo
        full_model.compile(loss='categorical_crossentropy',
                          optimizer= keras.optimizers.SGD(learning_rate=lr_schedule),
                          metrics=['accuracy',Recall(),'AUC'])

        # Treinando o modelo com os dados de treino e teste
        history = full_model.fit(
            train_generator,
            batch_size=len(train_generator),
            validation_data = val_generator,
            epochs = 25,
            callbacks=[callback,reduce_lr_on_plateau]
        )

        # list all data in history
        print(history.history.keys())
        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
        '''
        # summarize history for auc
        plt.plot(history.history['auc'])
        plt.plot(history.history['val_auc'])
        plt.title('Model auc')
        plt.ylabel('auc')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
        '''
        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()

        #dados de teste
        path_test = folder+filename+'/test/'
        test_generator =  test_datagen.flow_from_directory(
            path_test,
            target_size = (256, 256),
            shuffle = False,
            batch_size= batch_size,
            class_mode='categorical' 

            )

        from sklearn.metrics import confusion_matrix, classification_report
        Y_pred = full_model.predict(test_generator)
        y_pred = np.argmax(Y_pred, axis=1)

        cmvgg = confusion_matrix(test_generator.classes, y_pred )

        print(classification_report(test_generator.classes, y_pred))

        class_names = ['Benign','Malignant']
        report = classification_report(test_generator.classes, y_pred,target_names=class_names, output_dict=True)
        df = pd.DataFrame(report).transpose()
        df_ = df[['precision', 'recall','f1-score']]
        df_ = df_.loc[["Benign", "Malignant","macro avg"]]
        df_.to_csv(filename+'_resnet50.csv')
        df_.head(10)

        from sklearn.metrics import confusion_matrix, accuracy_score
        import seaborn as sns

        cm = confusion_matrix(test_generator.classes, y_pred)
        #cm = [[24,11],[1,34]]
        plt.figure(figsize=(5,5))
        x_axis_labels = ['Benign','Malignant'] # labels for x-axis
        y_axis_labels = ['Benign','Malignant'] # labels for y-axis
        ax = sns.heatmap(cm, annot=True, fmt="d",xticklabels=x_axis_labels, yticklabels=y_axis_labels)
        sns.set(font_scale=1.8)
        plt.title('Confusion matrix RESNET50')
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')

        from sklearn.metrics import roc_curve,auc, roc_auc_score,confusion_matrix, classification_report

        fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_generator.classes, y_pred)
        auc_keras = auc(fpr_keras, tpr_keras)

        plt.figure()
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')
        plt.show()